In [39]:
import os
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from subprocess import call

league = 'spanish'

df = None
files = os.listdir(f'fulldata/{league}')
files.sort()
for file in files:
    year = int(file.strip('.csv'))
    df_year = pd.read_csv(f'fulldata/{league}/' + file
                          # skiprows=1,
                          # index_col=None,
                          # names=['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']
                          )
    df_year.reset_index(drop=True, inplace=True)
    df_year['Year'] = year
    df_year['Match'] = df_year.index + 1

    if df is None:
        df = df_year
    else:
        df = df.append(df_year, ignore_index=True, sort=False)

# print(len(df))
# print(df.shape)

# remove unused columns
df_league = None
df.reset_index(inplace=True)
# df = df[['Year', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR',
#          "B365H", "B365D", "B365A"]]

predict_year = 2015
train_year = 2011
teams = ['Ath Madrid']
# teams = np.unique(df.loc[df['Year'] == predict_year, 'HomeTeam'].values)
teams.sort()
for team in teams:

    df_team = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    X = pd.DataFrame(
        data={
            'Year': df_team['Year'],
            'Date': df_team['Date'],
            'Team': team,
            'HomeMatch': df_team['HomeTeam'] == team
        }
    )
    X['Opponent'] = np.where(X['HomeMatch'], df_team['AwayTeam'], df_team['HomeTeam'])
    # X['HalfTimeGoals'] = np.where(X['HomeMatch'], df_team['HTHG'], df_team['HTAG'])
    # X['HalfTimeOpponentGoals'] = np.where(X['HomeMatch'], df_team['HTAG'], df_team['HTHG'])
    # X['HalfTimeLead'] = X['HalfTimeGoals'] > X['HalfTimeOpponentGoals']
    # X['HalfTimeLeadMoreThanTwo'] = (X['HalfTimeGoals'] - X['HalfTimeOpponentGoals']) > 2
    # X['FullTimeGoals'] = np.where(X['HomeMatch'], ath_madrid['FTHG'], ath_madrid['FTAG'])
    # X['FullTimeOpponentGoals'] = np.where(X['HomeMatch'], ath_madrid['FTAG'], ath_madrid['FTHG'])
    X['FTR'] = df_team['FTR']
    X['Won'] = np.where(X['HomeMatch'], df_team['FTR'] == 'H', df_team['FTR'] == 'A')
    X['Draw'] = df_team['FTR'] == 'D'
    X['Lost'] = np.where(X['HomeMatch'], df_team['FTR'] == 'A', df_team['FTR'] == 'H')
    X['Result'] = np.where(X['Won'], 'Win', (np.where(X['Lost'], 'Lose', 'Draw')))
    # X['SumGoals'] = X.groupby('Opponent')['FullTimeGoals'].transform(sum)
    X['B365Max'] = np.maximum(np.maximum(df_team['B365H'], df_team['B365A']), df_team['B365D'])
    X['B365Min'] = np.minimum(np.minimum(df_team['B365H'], df_team['B365A']), df_team['B365D'])
    X['B365Say'] = np.where(X['HomeMatch'],
                    # home match
                    np.where(X['B365Max'] == df_team['B365H'], -1,
                             np.where(X['B365Max'] == df_team['B365A'], 1,
                                      0)),
                    # away match
                    np.where(X['B365Max'] == df_team['B365H'], 1,
                             np.where(X['B365Max'] == df_team['B365A'], -1,
                                      0))
                    )
    X['B365Diff'] = np.where(X['B365Say'] == 1, X['B365Max'] - X['B365Min'], X['B365Min'] - X['B365Max'])

    # find number of times won against this opponent in last 5 meetings
    for key, groupByOpponent in X.groupby('Opponent'):
        # keep index as new a column, will be restored and assigned back to X later
        idx = groupByOpponent.index

        # make match day an index because rolling need an index date
        xx = groupByOpponent.set_index('Date')
        xx['idx'] = idx

        # shift to exclude self
        xx['Last5AgainstThisOpponentWon'] = xx['Won'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['Last5AgainstThisOpponentDraw'] = xx['Draw'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)

        xx['Last3AgainstThisOpponentWon'] = xx['Won'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['Last3AgainstThisOpponentDraw'] = xx['Draw'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)

        xx['LastAgainstThisOpponentWon'] = xx['Won'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['LastAgainstThisOpponentDraw'] = xx['Draw'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)

        # restore index
        xx = xx.set_index('idx')

        # assign back to the big dataframe
        X.loc[xx.index, 'Last5AgainstThisOpponentWon'] = xx['Last5AgainstThisOpponentWon']
        X.loc[xx.index, 'Last5AgainstThisOpponentDraw'] = xx['Last5AgainstThisOpponentDraw']
        X.loc[xx.index, 'Last3AgainstThisOpponentWon'] = xx['Last3AgainstThisOpponentWon']
        X.loc[xx.index, 'Last3AgainstThisOpponentDraw'] = xx['Last3AgainstThisOpponentDraw']
        X.loc[xx.index, 'LastAgainstThisOpponentWon'] = xx['LastAgainstThisOpponentWon']
        X.loc[xx.index, 'LastAgainstThisOpponentDraw'] = xx['LastAgainstThisOpponentDraw']

    # find recent forms
    idx = X.index
    xx = X.set_index('Date')
    xx['idx'] = idx
    xx['Last5Won'] = xx['Won'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['Last5Draw'] = xx['Draw'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['Last3Won'] = xx['Won'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['Last3Draw'] = xx['Draw'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['LastWon'] = xx['Won'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['LastDraw'] = xx['Draw'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['LastMatch'] = np.where(xx['LastWon'] == 1, 1, np.where(xx['LastDraw'] == 1, 0, -1))
    
    # restore index
    xx = xx.set_index('idx')

    # assign back to the big dataframe
    X.loc[xx.index, 'Last5Won'] = xx['Last5Won']
    X.loc[xx.index, 'Last5Draw'] = xx['Last5Draw']
    X.loc[xx.index, 'Last3Won'] = xx['Last3Won']
    X.loc[xx.index, 'Last3Draw'] = xx['Last3Draw']
    X.loc[xx.index, 'LastWon'] = xx['LastWon']
    X.loc[xx.index, 'LastDraw'] = xx['LastDraw']
    X.loc[xx.index, 'LastMatch'] = xx['LastMatch']

    # replace nan with 0
    # TODO: better way to handle nan
    X.loc[np.isnan(X['Last5AgainstThisOpponentWon']), 'Last5AgainstThisOpponentWon'] = 0
    X.loc[np.isnan(X['Last5AgainstThisOpponentDraw']), 'Last5AgainstThisOpponentDraw'] = 0
    X.loc[np.isnan(X['Last3AgainstThisOpponentWon']), 'Last3AgainstThisOpponentWon'] = 0
    X.loc[np.isnan(X['Last3AgainstThisOpponentDraw']), 'Last3AgainstThisOpponentDraw'] = 0
    X.loc[np.isnan(X['LastAgainstThisOpponentWon']), 'LastAgainstThisOpponentWon'] = 0
    X.loc[np.isnan(X['LastAgainstThisOpponentDraw']), 'LastAgainstThisOpponentDraw'] = 0
    #     X.loc[np.isnan(X['Last5Won']), 'Last5Won'] = 0
    #     X.loc[np.isnan(X['Last5Draw']), 'Last5Draw'] = 0
    #     X.loc[np.isnan(X['Last3Won']), 'Last3Won'] = 0
    #     X.loc[np.isnan(X['Last3Draw']), 'Last3Draw'] = 0
    #     X.loc[np.isnan(X['LastWon']), 'LastWon'] = 0
    #     X.loc[np.isnan(X['LastDraw']), 'LastDraw'] = 0

    # restrict training data (too old data may not be irrelevance)
    X = X.loc[X['Year'] >= train_year]
#     Y = X[['Opponent', 'Result']]
    Y = X[['Result']]
    
#     X_full = X

    # remove duplicate features
    del X['LastWon']
    del X['LastDraw']

    # prevent future leaks
    result = pd.DataFrame(X['Result'])
    del X['Result']
    del X['Lost']
    del X['Draw']
    del X['Won']
    del X['FTR']
    del X['Date']
    del X['Opponent']
    del X['Team']
    del X['B365Max']
    del X['B365Min']

    # split data into train - test sets
    x_train = X[(X['Year'] < predict_year)]
    y_train = Y[(X['Year'] < predict_year)]
    x_test = X[(X['Year'] >= predict_year)]
    y_test = Y[(X['Year'] >= predict_year)]
    
    

    # split prediction by opponent
    # construct decision tree
#     x_train_opponents = x_train.groupby('Opponent')
#     y_train_opponents = y_train.groupby('Opponent')
#     x_test_opponents = x_test.groupby('Opponent')
#     y_test_opponents = y_test.groupby('Opponent')
    #     x_test_teams = X_test_opponents.groups.keys()
#     x_test_teams = ['Villarreal']
#     X['Predict'] = ''
    #     os.makedirs(f'decision_tree/{league}/{predict_year}/{team}/', exist_ok=True)
    #     for key, x_train_opponent in x_train_opponents:
#     for key in x_test_teams:
        #         if key not in x_test_teams:
        #             continue
#         x_train_opponent = x_train_opponents.get_group(key)
#         x_test_opponent = x_test_opponents.get_group(key)
#         y_train_opponent = y_train_opponents.get_group(key)
#         y_test_opponent = y_test_opponents.get_group(key)

#         del y_train_opponent['Opponent']
#         del y_test_opponent['Opponent']
#         del x_train_opponent['Opponent']
#         del x_test_opponent['Opponent']
#         del x_train_opponent['Year']
#         del x_test_opponent['Year']
#         del x_train_opponent['Team']
#         del x_test_opponent['Team']

#         clf = DecisionTreeClassifier(
#             criterion="entropy",
#             random_state=100,
#             min_samples_leaf=3
#         )
#         clf.fit(x_train_opponent, y_train_opponent)


In [40]:
x_train

,Year,HomeMatch,B365Say,B365Diff,Last5AgainstThisOpponentWon,Last5AgainstThisOpponentDraw,Last3AgainstThisOpponentWon,Last3AgainstThisOpponentDraw,LastAgainstThisOpponentWon,LastAgainstThisOpponentDraw,Last5Won,Last5Draw,Last3Won,Last3Draw,LastMatch
2284,2011,True,1,4.97,3.0,0.0,3.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,1
2292,2011,False,-1,-1.84,2.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,0
2306,2011,True,1,6.06,0.0,3.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,-1
2313,2011,True,1,6.10,3.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,1.0,1.0,1
2321,2011,False,-1,-11.80,2.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,1
2335,2011,True,1,1.84,1.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,2.0,0.0,-1
2342,2011,False,1,1.40,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0
2354,2011,True,1,6.06,2.0,1.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,2.0,0
2368,2011,False,0,-1.02,3.0,0.0,2.0,0.0,0.0,0.0,1.0,3.0,0.0,3.0,0
2374,2011,True,1,6.60,3.0,1.0,2.0,1.0,1.0,0.0,0.0,3.0,0.0,2.0,-1


In [41]:
y_train

,Result
2284,Draw
2292,Lose
2306,Win
2313,Win
2321,Lose
2335,Draw
2342,Draw
2354,Draw
2368,Lose
2374,Win


In [42]:
clf = DecisionTreeClassifier(
    criterion="entropy",
    random_state=100,
    min_samples_leaf=3
)
clf.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [43]:
# in-sample test
y_insample_pred = clf.predict(x_train)
accuracy_score(y_train, y_insample_pred)*100

86.8421052631579

In [44]:
# out-sample test
y_pred = clf.predict(x_test)
accuracy_score(y_test, y_pred)*100

47.368421052631575

In [45]:
clf = RandomForestClassifier(n_estimators=10)
clf.fit(x_train, y_train['Result'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
# in-sample test
y_insample_pred = clf.predict(x_train)
y_insample_pred

array(['Draw', 'Lose', 'Win', 'Win', 'Lose', 'Draw', 'Draw', 'Draw',
       'Lose', 'Win', 'Lose', 'Win', 'Lose', 'Win', 'Lose', 'Lose',
       'Draw', 'Win', 'Win', 'Win', 'Draw', 'Draw', 'Draw', 'Lose',
       'Draw', 'Win', 'Lose', 'Win', 'Lose', 'Win', 'Lose', 'Lose', 'Win',
       'Win', 'Draw', 'Draw', 'Win', 'Win', 'Draw', 'Win', 'Win', 'Win',
       'Win', 'Win', 'Win', 'Win', 'Win', 'Lose', 'Win', 'Win', 'Win',
       'Lose', 'Win', 'Lose', 'Win', 'Draw', 'Win', 'Win', 'Lose', 'Win',
       'Lose', 'Win', 'Win', 'Draw', 'Lose', 'Win', 'Draw', 'Draw', 'Win',
       'Win', 'Lose', 'Draw', 'Draw', 'Lose', 'Draw', 'Win', 'Win', 'Win',
       'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Lose', 'Win', 'Win',
       'Win', 'Draw', 'Win', 'Win', 'Win', 'Win', 'Win', 'Draw', 'Draw',
       'Win', 'Win', 'Lose', 'Win', 'Lose', 'Draw', 'Win', 'Win', 'Win',
       'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Draw', 'Draw',
       'Draw', 'Win', 'Win', 'Draw', 'Win', 'Win', 'Lose', 'Win

In [47]:
y_train

,Result
2284,Draw
2292,Lose
2306,Win
2313,Win
2321,Lose
2335,Draw
2342,Draw
2354,Draw
2368,Lose
2374,Win


In [48]:
accuracy_score(y_train, y_insample_pred)*100

98.02631578947368

In [49]:
# out-sample test
y_pred = clf.predict(x_test)
y_pred

array(['Win', 'Win', 'Lose', 'Win', 'Win', 'Draw', 'Win', 'Draw', 'Win',
       'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Win',
       'Win', 'Win', 'Win', 'Lose', 'Win', 'Draw', 'Win', 'Win', 'Win',
       'Win', 'Win', 'Lose', 'Win', 'Draw', 'Win', 'Lose', 'Win', 'Win',
       'Lose', 'Win', 'Win', 'Draw', 'Draw', 'Win', 'Lose', 'Win', 'Win',
       'Win', 'Win', 'Win', 'Lose', 'Win', 'Win', 'Win', 'Draw', 'Win',
       'Win', 'Win', 'Draw', 'Win', 'Win', 'Win', 'Draw', 'Draw', 'Win',
       'Draw', 'Lose', 'Win', 'Lose', 'Win', 'Win', 'Win', 'Win', 'Win',
       'Win', 'Win', 'Win', 'Win', 'Win', 'Draw', 'Lose', 'Win', 'Draw',
       'Win', 'Win', 'Lose', 'Draw', 'Win', 'Win', 'Win', 'Win', 'Draw',
       'Win', 'Win', 'Lose', 'Win', 'Draw', 'Win', 'Win', 'Win', 'Win',
       'Win', 'Lose', 'Win', 'Lose', 'Win', 'Win', 'Win', 'Lose', 'Win',
       'Lose', 'Win', 'Win', 'Win', 'Draw', 'Draw'], dtype=object)

In [50]:
y_test

,Result
3801,Win
3818,Win
3821,Lose
3831,Win
3840,Win
3855,Lose
3867,Draw
3877,Win
3885,Win
3890,Draw


In [51]:
accuracy_score(y_test, y_pred)*100

60.526315789473685